# Benchmarking

## 1. Prepare working environment

In [1]:
import timm
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from PIL import Image
import pandas as pd
import os

from scripts import Worker

In [2]:
!nvidia-smi

Wed May 17 08:33:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  Off |
|  0%   36C    P8    20W / 450W |      0MiB / 24564MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


## 2. Datasets and visulisation

In [4]:
class MyDataset(Dataset):
    def __init__(self, data_dir, transforms=None):
        self.data_dir = data_dir
        self.data_info = self.get_img_info(data_dir)
        self.transforms = transforms

    def __getitem__(self, item):
        path_img, label = self.data_info.iloc[item][1:3]
        label = int(label)
        path_img = os.path.join(self.data_dir, path_img)
        image = Image.open(path_img).convert('RGB') # Gray scale is enough for logic interpretation
        # 使用定义好的transforms，对数据进行处理
        if self.transforms is not None:
            image = self.transforms(image)

        return image, label

    def __len__(self):
        return len(self.data_info)
    
    def get_img_info(self, data_dir):
        path_dir = os.path.join(data_dir, 'label.csv')
        return pd.read_csv(path_dir)

In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) 

classes = (0, 1)
model_names = ['xception','densenet201','resnetv2_50', 'vgg16', 'darknet53', 'mobilenetv3_large_100', 'inception_resnet_v2', 'nasnetalarge', 'efficientnetv2_rw_m', 'convnext_xlarge_384_in22ft1k']

In [6]:
for dataset_folder in ["dataset02", "dataset03", "dataset04", "dataset05"]:
    trainset = MyDataset(f"data/train/{dataset_folder}", transform)
    testset = MyDataset(f"data/test/{dataset_folder}", transform)

    # trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=8)

    # testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True, num_workers=8)

    exp_name = f"{dataset_folder}_benchmarking"

    for model_name in model_names:
        print(f"Log: {model_name}")
        model = timm.create_model(model_name, num_classes=2, pretrained=True)
        worker = Worker(exp_name, model_name, model, trainloader, testloader, epochs=1, lr=1e-3)
        worker.train()

Log: xception
state dict found: ./models/dataset02_benchmarking/xception.pt
Log: densenet201
state dict found: ./models/dataset02_benchmarking/densenet201.pt
Log: resnetv2_50
state dict found: ./models/dataset02_benchmarking/resnetv2_50.pt
Log: vgg16
state dict found: ./models/dataset02_benchmarking/vgg16.pt
Log: darknet53
state dict found: ./models/dataset02_benchmarking/darknet53.pt
Log: mobilenetv3_large_100
state dict found: ./models/dataset02_benchmarking/mobilenetv3_large_100.pt
Log: inception_resnet_v2
state dict found: ./models/dataset02_benchmarking/inception_resnet_v2.pt
Log: nasnetalarge
state dict found: ./models/dataset02_benchmarking/nasnetalarge.pt
Log: efficientnetv2_rw_m
state dict found: ./models/dataset02_benchmarking/efficientnetv2_rw_m.pt
Log: convnext_xlarge_384_in22ft1k
state dict found: ./models/dataset02_benchmarking/convnext_xlarge_384_in22ft1k.pt
Log: xception
state dict found: ./models/dataset03_benchmarking/xception.pt
Log: densenet201
state dict found: ./m

epoch,train_loss,valid_loss,accuracy,time
0,1.178393,1.031338,0.758500,00:10


epoch,train_loss,valid_loss,accuracy,time
0,0.332413,0.295760,0.865500,00:10


Log: inception_resnet_v2
state dict found: ./models/dataset03_benchmarking/inception_resnet_v2.pt
Log: nasnetalarge
state dict found: ./models/dataset03_benchmarking/nasnetalarge.pt
Log: efficientnetv2_rw_m
state dict found: ./models/dataset03_benchmarking/efficientnetv2_rw_m.pt
Log: convnext_xlarge_384_in22ft1k
state dict found: ./models/dataset03_benchmarking/convnext_xlarge_384_in22ft1k.pt
Log: xception
state dict found: ./models/dataset04_benchmarking/xception.pt
Log: densenet201
state dict found: ./models/dataset04_benchmarking/densenet201.pt
Log: resnetv2_50


epoch,train_loss,valid_loss,accuracy,time
0,0.331258,0.287560,0.836500,00:10


epoch,train_loss,valid_loss,accuracy,time
0,0.116117,0.079961,0.979000,00:10


Log: vgg16
state dict found: ./models/dataset04_benchmarking/vgg16.pt
Log: darknet53
state dict found: ./models/dataset04_benchmarking/darknet53.pt
Log: mobilenetv3_large_100
state dict found: ./models/dataset04_benchmarking/mobilenetv3_large_100.pt
Log: inception_resnet_v2
state dict found: ./models/dataset04_benchmarking/inception_resnet_v2.pt
Log: nasnetalarge
state dict found: ./models/dataset04_benchmarking/nasnetalarge.pt
Log: efficientnetv2_rw_m
state dict found: ./models/dataset04_benchmarking/efficientnetv2_rw_m.pt
Log: convnext_xlarge_384_in22ft1k
state dict found: ./models/dataset04_benchmarking/convnext_xlarge_384_in22ft1k.pt
Log: xception
state dict found: ./models/dataset05_benchmarking/xception.pt
Log: densenet201
state dict found: ./models/dataset05_benchmarking/densenet201.pt
Log: resnetv2_50


epoch,train_loss,valid_loss,accuracy,time
0,0.471160,0.427172,0.786500,00:10


epoch,train_loss,valid_loss,accuracy,time
0,0.227292,0.173833,0.935500,00:10


Log: vgg16
state dict found: ./models/dataset05_benchmarking/vgg16.pt
Log: darknet53
state dict found: ./models/dataset05_benchmarking/darknet53.pt
Log: mobilenetv3_large_100
state dict found: ./models/dataset05_benchmarking/mobilenetv3_large_100.pt
Log: inception_resnet_v2
state dict found: ./models/dataset05_benchmarking/inception_resnet_v2.pt
Log: nasnetalarge
state dict found: ./models/dataset05_benchmarking/nasnetalarge.pt
Log: efficientnetv2_rw_m
state dict found: ./models/dataset05_benchmarking/efficientnetv2_rw_m.pt
Log: convnext_xlarge_384_in22ft1k
state dict found: ./models/dataset05_benchmarking/convnext_xlarge_384_in22ft1k.pt
